In [9]:
import pandas as pd
import datetime
import requests

# Reference for how to call Spotify API:
# https://stmorse.github.io/journal/spotify-api.html

In [10]:
# To create an age column later
now = datetime.datetime.now()

In [11]:
# Matt's Spotify api call info
CLIENT_ID = 'a8a69aee68c7407aadc7eb1ba17b90b5'
CLIENT_SECRET = '5694ed52d6184a95816b00272d296cf3'

In [12]:
# Generic Spotify
AUTH_URL = 'https://accounts.spotify.com/api/token'
CLIENT_ID = 'a8a69aee68c7407aadc7eb1ba17b90b5'
CLIENT_SECRET = '5694ed52d6184a95816b00272d296cf3'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [13]:
taylor = '06HL4z0CvFAxyc27GXpf02'
drake = '3TVXtAsR1Inumwj472S9r4'

In [26]:
artist_id = taylor

In [27]:
# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums',
                 headers=headers,
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [28]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']
    album_name_first = album_name.split(' ')[0]

    # this takes a few seconds so let's keep track of progress


    if album_name_first not in albums:
        print(album_name)
        # pull all tracks from this album
        r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks',
                         headers=headers)
        tracks = r.json()['items']

        for track in tracks:
            # get audio features (key, liveness, danceability, ...)
            f = requests.get(BASE_URL + 'audio-features/' + track['id'],
                             headers=headers)
            f = f.json()

            # combine with album info
            f.update({
                'track_name': track['name'],
                'album_name': album_name,
                'release_date': album['release_date'],
                'album_id': album['id']
            })

            data.append(f)
            albums.append(album_name_first)

Midnights (3am Edition)
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
Lover
Taylor Swift Karaoke: reputation
reputation
1989
Speak Now (Japanese Version)


In [29]:
df = pd.DataFrame(data)
df['release_date'] = pd.to_datetime(df['release_date'])
df = df.sort_values(by='release_date')

df['age'] = df['release_date'].apply(lambda x : (now - x).days)
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,release_date,album_id,age
190,0.546,0.809,4,-3.621,1,0.0410,0.00487,0.000372,0.0437,0.649,...,spotify:track:0zcJ2N8nyYXF0rkO9MOiUU,https://api.spotify.com/v1/tracks/0zcJ2N8nyYXF...,https://api.spotify.com/v1/audio-analysis/0zcJ...,266480,4,The Story Of Us,Speak Now (Japanese Version),2010-10-25,4ErTrymYK8VIBQR8J8Hjy1,4424
175,0.696,0.768,7,-3.863,1,0.0308,0.00461,0.000001,0.1010,0.692,...,spotify:track:4wLBRunveZpcCuivglwFPa,https://api.spotify.com/v1/tracks/4wLBRunveZpc...,https://api.spotify.com/v1/audio-analysis/4wLB...,230547,4,Mine - POP Mix,Speak Now (Japanese Version),2010-10-25,4ErTrymYK8VIBQR8J8Hjy1,4424
176,0.608,0.785,5,-2.976,1,0.0311,0.03870,0.000000,0.1580,0.376,...,spotify:track:2lCii9ugWIOMGLKO5dj5WP,https://api.spotify.com/v1/tracks/2lCii9ugWIOM...,https://api.spotify.com/v1/audio-analysis/2lCi...,260947,4,Sparks Fly,Speak Now (Japanese Version),2010-10-25,4ErTrymYK8VIBQR8J8Hjy1,4424
177,0.517,0.606,2,-5.797,1,0.0289,0.02020,0.000000,0.3240,0.296,...,spotify:track:4moQek8ZD57EpFHby1uyOu,https://api.spotify.com/v1/tracks/4moQek8ZD57E...,https://api.spotify.com/v1/audio-analysis/4moQ...,293040,4,Back To December,Speak Now (Japanese Version),2010-10-25,4ErTrymYK8VIBQR8J8Hjy1,4424
178,0.708,0.601,7,-3.750,1,0.0306,0.10100,0.000000,0.0979,0.742,...,spotify:track:7rkOO7Zbr9WimwXQ7Ojfw0,https://api.spotify.com/v1/tracks/7rkOO7Zbr9Wi...,https://api.spotify.com/v1/audio-analysis/7rkO...,240773,4,Speak Now,Speak Now (Japanese Version),2010-10-25,4ErTrymYK8VIBQR8J8Hjy1,4424


In [30]:
# Grabbing specific columns
graph_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
                 'track_name', 'age', 'album_name']

df = df.loc[:,graph_columns]
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_name,age,album_name
190,0.546,0.809,4,-3.621,1,0.0410,0.00487,0.000372,0.0437,0.6490,139.910,266480,The Story Of Us,4424,Speak Now (Japanese Version)
175,0.696,0.768,7,-3.863,1,0.0308,0.00461,0.000001,0.1010,0.6920,121.050,230547,Mine - POP Mix,4424,Speak Now (Japanese Version)
176,0.608,0.785,5,-2.976,1,0.0311,0.03870,0.000000,0.1580,0.3760,114.985,260947,Sparks Fly,4424,Speak Now (Japanese Version)
177,0.517,0.606,2,-5.797,1,0.0289,0.02020,0.000000,0.3240,0.2960,141.929,293040,Back To December,4424,Speak Now (Japanese Version)
178,0.708,0.601,7,-3.750,1,0.0306,0.10100,0.000000,0.0979,0.7420,118.962,240773,Speak Now,4424,Speak Now (Japanese Version)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.675,0.466,11,-9.738,0,0.2010,0.36100,0.000000,0.1100,0.3470,140.864,148781,Glitch,44,Midnights (3am Edition)
18,0.483,0.840,7,-6.508,1,0.1190,0.43000,0.000575,0.1460,0.5500,157.979,260362,"Would've, Could've, Should've",44,Midnights (3am Edition)
19,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,225194,Dear Reader,44,Midnights (3am Edition)
10,0.649,0.622,8,-7.054,1,0.0582,0.07400,0.000000,0.5940,0.1060,90.024,204852,Karma,44,Midnights (3am Edition)


In [31]:
if artist_id == taylor:
    taylor_rename_df = {
        'Speak Now (Japanese Version)' : 'Speak Now' ,
        '1989' : '1989' ,
        'reputation' : 'Reputation' ,
        'Lover' : 'Lover' ,
        'folklore (deluxe version)' : 'Folklore' ,
        'evermore (deluxe version)' : 'Evermore' ,
        "Fearless (Taylor's Version)" : 'Fearless' ,
        "Red (Taylor's Version)" : 'Red' ,
        'Midnights (3am Edition)' : 'Taylor Swift Album',
        'Taylor Swift Karaoke: reputation' : 'x',
        'folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]' : 'xx'
    }
    df['album_name'] = df['album_name'].apply(lambda x: taylor_rename_df[x])

In [32]:
if artist_id == drake:
    drake_rename_df = {
        "Her Loss": "a",
        "Honestly, Nevermind": "b",
        "Certified Lover Boy": "Certified Lover Boy",
        "Dark Lane Demo Tapes": "c",
        "Care Package": "d",
        "So Far Gone": "e",
        "Scorpion": "Scorpion",
        "More Life": "f",
        "Views": "Views",
        "What A Time To Be Alive": "g",
        "If You're Reading This It's Too Late": "h",
        "Nothing Was The Same (Deluxe)": "Nothing Was the Same",
        "Take Care (Deluxe)": "Take Care",
        "Thank Me Later (Int'l Version)": "Thank me Later"
    }
    df['album_name'] = df['album_name'].apply(lambda x: drake_rename_df[x])

In [33]:
df['album_name']

190             Speak Now
175             Speak Now
176             Speak Now
177             Speak Now
178             Speak Now
              ...        
17     Taylor Swift Album
18     Taylor Swift Album
19     Taylor Swift Album
10     Taylor Swift Album
0      Taylor Swift Album
Name: album_name, Length: 191, dtype: object

In [34]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_name,age,album_name
190,0.546,0.809,4,-3.621,1,0.0410,0.00487,0.000372,0.0437,0.6490,139.910,266480,The Story Of Us,4424,Speak Now
175,0.696,0.768,7,-3.863,1,0.0308,0.00461,0.000001,0.1010,0.6920,121.050,230547,Mine - POP Mix,4424,Speak Now
176,0.608,0.785,5,-2.976,1,0.0311,0.03870,0.000000,0.1580,0.3760,114.985,260947,Sparks Fly,4424,Speak Now
177,0.517,0.606,2,-5.797,1,0.0289,0.02020,0.000000,0.3240,0.2960,141.929,293040,Back To December,4424,Speak Now
178,0.708,0.601,7,-3.750,1,0.0306,0.10100,0.000000,0.0979,0.7420,118.962,240773,Speak Now,4424,Speak Now
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.675,0.466,11,-9.738,0,0.2010,0.36100,0.000000,0.1100,0.3470,140.864,148781,Glitch,44,Taylor Swift Album
18,0.483,0.840,7,-6.508,1,0.1190,0.43000,0.000575,0.1460,0.5500,157.979,260362,"Would've, Could've, Should've",44,Taylor Swift Album
19,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,225194,Dear Reader,44,Taylor Swift Album
10,0.649,0.622,8,-7.054,1,0.0582,0.07400,0.000000,0.5940,0.1060,90.024,204852,Karma,44,Taylor Swift Album


In [35]:
# Standardizing features on the range 0-1
stand_columns = ['key', 'loudness', 'tempo', 'age']

for col in stand_columns:
    min = df[col].min()
    df[col] = df[col]+abs(min)
    max = df[col].max()
    df[col] = df[col]/max

df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_name,age,album_name
190,0.546,0.809,0.363636,0.928509,1,0.0410,0.00487,0.000372,0.0437,0.6490,0.751280,266480,The Story Of Us,1.000000,Speak Now
175,0.696,0.768,0.636364,0.910855,1,0.0308,0.00461,0.000001,0.1010,0.6920,0.683304,230547,Mine - POP Mix,1.000000,Speak Now
176,0.608,0.785,0.454545,0.975562,1,0.0311,0.03870,0.000000,0.1580,0.3760,0.661444,260947,Sparks Fly,1.000000,Speak Now
177,0.517,0.606,0.181818,0.769769,1,0.0289,0.02020,0.000000,0.3240,0.2960,0.758556,293040,Back To December,1.000000,Speak Now
178,0.708,0.601,0.636364,0.919098,1,0.0306,0.10100,0.000000,0.0979,0.7420,0.675778,240773,Speak Now,1.000000,Speak Now
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.675,0.466,1.000000,0.482273,0,0.2010,0.36100,0.000000,0.1100,0.3470,0.754718,148781,Glitch,0.019696,Taylor Swift Album
18,0.483,0.840,0.636364,0.717902,1,0.1190,0.43000,0.000575,0.1460,0.5500,0.816404,260362,"Would've, Could've, Should've",0.019696,Taylor Swift Album
19,0.562,0.388,0.000000,0.310840,1,0.0705,0.48100,0.001450,0.1170,0.1590,0.635357,225194,Dear Reader,0.019696,Taylor Swift Album
10,0.649,0.622,0.727273,0.678071,1,0.0582,0.07400,0.000000,0.5940,0.1060,0.571479,204852,Karma,0.019696,Taylor Swift Album


In [36]:
#Export to csv
if artist_id == taylor:
    df.to_csv('taylor_spotify.csv')

if artist_id == drake:
    df.to_csv('drake_spotify.csv')